**Objective:**  **Explore the possibility of building a minimally viable product (MVP) model to predict the likelihood of a restaurant receiving a Critical violation.**

**The following jupyter notebook contains the code for modeling and step-by step explaination  for each step**

Packages needed for modeling






*   Install **Pandas Profiling** package for initial Data Exploration
*   Install **Category Encoders** package for performing target encoding of categorical variable





### 0) Install & import necessary packages





In [3]:
!pip install pandas-profiling[notebook]
! pip install category_encoders 

Import all necessary packages for the model building

In [0]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss
from matplotlib import pyplot
from IPython.core.display import display, HTML
from sklearn.metrics import log_loss

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### **1) Read the data from the source**

**Function Name**: read_data

**Parameter**: PATH: Path where the data file is stored. I have stored the data file on Google Drive and my PATH variable contains the path of the file on Google Drive

**Function Description**: Read the excel from the path

In [0]:
def read_data(PATH):
  data = pd.read_excel(PATH)
  return data


In [0]:
#Read the data from the path
PATH="/content/drive/My Drive/Restaurants_Inspection/RESTAURANT_INSPECTIONS_PROCESSED.xlsx"
data=read_data(PATH)

### **2) Initial Data Exploration**

In order to do a quick thorough exploration of the dataset and identify patterns in them I am using the **pandas profiling  package** that generates an exhaustive report of all the features of the dataset 

In [16]:
#Generate Initial report on the dataset using pandas profile reporting
profile = ProfileReport(data)
a=profile.to_html()
display(HTML(a))


/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,36
Number of observations,31419
Total Missing (%),2.2%
Total size in memory,8.6 MiB
Average record size in memory,288.0 B
Numeric,6
Categorical,25
Boolean,0
Date,1
Text (Unique),1
Rejected,3


### **3) Data Cleaning**

**Function Name** : clean_explore


**Parameter** : r_data : original pandas dataframe

**Function Description** : 

*   Since our objective is to estimate the likelihood of the restaurant receiving First Violation Type as 'Critical', I have encoded this value as '1' and other Violation Types as '0'.So that for every row , we can predict the probabilty of Violation Type as Critical based on the attributes of the restaurant.
*   This function also replaces NaN values in one the 'INSPECTION_TIME' time columns with default value '01/01/1900 12:00:00 AM' which is important for our analysis further. 
*   For the other NaN values in categorical columns ,I replace them with 'Unkown' creating a new category for each of the columns. The purpose of doing this is to preserve information from the restaurnts that have few missing values in them.






In [0]:
def clean_explore(r_data):
  r_data.loc[r_data.FIRST_VIOLATION_TYPE=='Critical','FIRST_VIOLATION_TYPE']=1
  r_data.loc[r_data.FIRST_VIOLATION_TYPE!=1,'FIRST_VIOLATION_TYPE']=0
  r_data['INSPECTION_TIME'].replace(np.nan,'01/01/1900 12:00:00 AM',inplace=True)
  r_data.replace(np.nan,'Unknown',inplace=True)
  return r_data

In [0]:
#Perform initial data cleaning and transformation
cleaned_data=clean_explore(data)

After looking at the data exploration report it can be infered that there are few columns that are not needed in the analysis which I am dropping from further consideration .

*   The columns 'SECOND_VIOLATION_TYPE','THIRD_VIOLATION_TYPE','SECOND_VIOLATION','THIRD_VIOLATION' and their descriptions are not needed in our analysis since we are interested in only predicting the likelihood of 'FIRST_VIOLATION_TYPE' as Critical.Hence,I have dropped these columns.
*   The columns 'LOCATION_LONGITUDE','LOCATION_LATITUDE','LAT_LONG_RAW' provide a very granular detail on the location of the restaurant which is not needed in our model building 
*   Similary the columns 'RECORD_UPDATED','EMPLOYEE_ID','RESTAURANT_PERMIT_NUMBER','STATE'(same value on all rows) add no value to our analysis hence I have dropped them







In [21]:
data=cleaned_data
columnsToBeDroped=['SECOND_VIOLATION_TYPE','THIRD_VIOLATION_TYPE','SECOND_VIOLATION',
                   'THIRD_VIOLATION','LOCATION_LONGITUDE','LOCATION_LATITUDE',
                   'LAT_LONG_RAW','RECORD_UPDATED','EMPLOYEE_ID',
                   'RESTAURANT_PERMIT_NUMBER','STATE','FIRST_VIOLATION_DS','SECOND_VIOLATION_DS',
                   'THIRD_VIOLATION_DS','DATE_CURRENT','INSPECTION_DATE_RAW','VIOLATIONS_RAW','RESTAURANT_NAME','ADDRESS']

if set(columnsToBeDroped).issubset(data.columns):
  data=data.drop(columns=columnsToBeDroped)
print(data.head())

  RESTAURANT_SERIAL_NUMBER  ... NUMBER_OF_VIOLATIONS
0                DA0224668  ...                    3
1                DA0228150  ...                    3
2                DA0349036  ...                   12
3                DA0356449  ...                   10
4                DA0417263  ...                    3

[5 rows x 17 columns]


This is my hypothesis that Inspection Date and Time in itself might not be correlated with Insepction results but the Quarter in which the inspection happens and the hour of the day might be important in determining the inspection results.

Example during rush hours at night some restaurants mightget carless in maintaining adequate hygine conditions and hence may result in critical violation.


Hence ,I have created Inspection Quarter and Inspection Time column in order to preserve information at a little higher level for each restaurants.



In [0]:
for i in range(data.shape[0]):
    a= data.loc[i,'INSPECTION_DATE'].month//3
    quarter = "Q"+str(a) 
    data.loc[i,'INSPECTION_QUARTER']=quarter
    if 'PM' in data.loc[i,'INSPECTION_TIME']:
      hour=int(data.loc[i,'INSPECTION_TIME'].split(' ')[1].split(':')[0])+12
      data.loc[i,'INSPECTION_TIME_HOUR']=str(hour)
    else:
      data.loc[i,'INSPECTION_TIME_HOUR']=str(data.loc[i,'INSPECTION_TIME']).split(' ')[1].split(':')[0]
data=data.drop(columns=['INSPECTION_DATE','INSPECTION_TIME'])


**Exploratory Report After Data Cleaning**

In [24]:
profile_cleaned = ProfileReport(data)
cleaned_report=profile_cleaned.to_html()
display(HTML(cleaned_report))


/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,17
Number of observations,31419
Total Missing (%),0.0%
Total size in memory,4.1 MiB
Average record size in memory,136.0 B
Numeric,4
Categorical,11
Boolean,1
Date,0
Text (Unique),1
Rejected,0


**Function Name**: train_test

**Function Paramter** : Cleaned Data

**Function Description** : Takes the cleaned data and separates the Feature variables from the target variable which is  ' FIRST_VIOLATION_TYPE' in our case  and then split the Data in Train Test split in 80:20 proportion.
This function retunrs the Train split and Test split of the feature and the target variables.

In [0]:
def train_test(data):
  data_label=data['FIRST_VIOLATION_TYPE']
  data_features=data.loc[:, data.columns != 'FIRST_VIOLATION_TYPE']
  X_train, X_test, y_train, y_test = train_test_split(data_features, data_label, test_size=0.2)
  print(X_train.shape, y_train.shape)
  print(X_test.shape, y_test.shape)
  return X_train, X_test, y_train, y_test


In [53]:
#Call the train test split
X_train, X_test, y_train, y_test = train_test(data)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(25135, 16) (25135,)
(6284, 16) (6284,)
(25135, 16) (25135,)
(6284, 16) (6284,)


### **4) Data Transformation**

Many features in our dataset are categorical variables which needs to be  encoded into numerical representation inorder to be used for our modeling algorithm. The most commonly used approaches for this is One Hot Encoding and Dummy encoding which created n and n-1 columns respectively ,where n is the number of distinct values in that column.Since our dataset contain many categorical value and they are with many number of distinct values ,I would not go ahead with one hot encoding or dummy encoding since it is going to explode my dataset size.


Hence, I  plan to use **Target Encoding** which encodes the categorical variable based on the target values in just one column.


**One important thing to keep in mind is that Target Encoding needs to be done on Train and Test set separately otherwise it creates the risk of Data Leakage.**

**Function Name**: Data Transformation

**Parameter**: One pandas Dataframe with Features and other with Labels

**Function Description**: 
This function encodes the categorical variables using Taregt Encoding method and encodes the numerical variables using MinMax Scalaer approach scaling the entire dataset in the range of 0 and 1.

In [0]:
def data_transformation(X,Y):
  te_data=ce.TargetEncoder(cols=['RESTAURANT_LOCATION','RESTAURANT_CATEGORY','CITY','ZIP','CURRENT_GRADE','INSPECTION_TYPE',
                                'INSPECTION_GRADE', 'PERMIT_STATUS', 'INSPECTION_RESULT',
                                'FIRST_VIOLATION','INSPECTION_QUARTER',
                                'INSPECTION_TIME_HOUR'])
  te = te_data.fit_transform(X,Y)
  te['CURRENT_DEMERITS']=(te['CURRENT_DEMERITS']-te['CURRENT_DEMERITS'].min())/te['CURRENT_DEMERITS'].max()
  te['INSPECTION_DEMERITS']=(te['INSPECTION_DEMERITS']-te['INSPECTION_DEMERITS'].min())/te['INSPECTION_DEMERITS'].max()
  te['NUMBER_OF_VIOLATIONS']=(te['NUMBER_OF_VIOLATIONS']-te['NUMBER_OF_VIOLATIONS'].min())/te['NUMBER_OF_VIOLATIONS'].max()
  te.index=X['RESTAURANT_SERIAL_NUMBER']
  
  return te

In [0]:
#Perform Data Transformation on Tarining and Test Data
te_train= data_transformation(X_train,y_train)
test_feature = data_transformation(X_test,y_test)

In [0]:
y_test.index = test_feature['RESTAURANT_SERIAL_NUMBER']
y_train.index=te_train['RESTAURANT_SERIAL_NUMBER']

In [0]:
te_train=te_train.drop(columns='RESTAURANT_SERIAL_NUMBER')
test_feature=test_feature.drop(columns='RESTAURANT_SERIAL_NUMBER')

### **5) Model Building**

**Function Name**: rf_gridModel

**Parameters** : Training Features and Labels

**Function Description:** This function fits a RandomForest Model with parameters mentioned in the 'param_grid'. Grid Search CV evaluates all combination that we provide.This will try out 1 * 3 * 3 * 3 * 4=108 combination to find out the best hyperparameter that provides the leaset log loss error.

In [0]:
def rf_gridModel(X,Y):
  # Create the parameter grid based on the results of random search 
  param_grid = {
    'bootstrap': [True],
    'max_depth': [70,80,90],
    'max_features': [10,12,14],
    'min_samples_leaf': [4, 5, 6],
    'n_estimators': [100, 200, 300, 1000]
  }
  # Create a based mode
  rf = RandomForestClassifier()
  # Instantiate the grid search model
  grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
  grid_search.fit(X, Y)
  return grid_search

In [34]:
model_grid = rf_gridModel(te_train,y_train)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed: 12.6min finished


### **6) Model Evaluation**

To evaluate the effectiveness of the model, we test it on the unseen test set.The measure that I have used to report error is '**log loss**' or **'cross entopy'** error is used to evaluate predicted probabilities error . Each predicted probability is compared to the actual class output value  and a score is calculated that penalizes the probability based on the distance from the expected value. The penalty is logarithmic, offering a small score for small differences (0.1 or 0.2) and enormous score for a large difference (0.9 or 1.0).

A model with perfect skill has a log loss score of 0.0.

**Function Name:** evaluate_p

**Parameter Name:** model , Test features and labels

**Function Description :** Evaluats the performance of the model by using the log-loss function.

In [0]:
def evaluate_p(model,X,Y):
  pred=model.predict_proba(X)[:,1]
  error= log_loss(Y, pred)
  return error

In [87]:
print("The parameter valus for the best model is as follows")
print(model_grid.best_params_)
best_grid = model_grid.best_estimator_
loss= evaluate_p(best_grid,test_feature,y_test)
print("The Log Loss of the best model is a follows")
print(loss)

The parameter valus for the best model is as follows
{'bootstrap': True, 'max_depth': 40, 'max_features': 10, 'min_samples_leaf': 4, 'n_estimators': 100}
The Log Loss of the best model is a follows
0.0055478103751101905


### **7) Interpretation**

Our RandomForest Model provides us with a metric that identifies the features that were most in determining the probability of the First Violation Type as Critical.Actionable Insights can be taken based on this feature importance metric as the most important features should be paid more intention in fututre to prevent Critical Violations

In [69]:
best_grid.feature_importances_

array([5.86295656e-03, 1.50318397e-07, 0.00000000e+00, 6.06207502e-04,
       4.70530746e-06, 0.00000000e+00, 8.88251499e-07, 1.14422861e-01,
       3.43243585e-02, 2.47209041e-06, 1.16775965e-04, 8.39915188e-01,
       4.72934098e-03, 2.22409873e-07, 1.38729837e-05])

In [85]:
feature_importances = pd.DataFrame(best_grid.feature_importances_*100,
                                   index = te_train.columns,
                                    columns=['Importance in Percentages']).sort_values('Importance in Percentages',ascending=False)
feature_importances

,Importance in Percentages
FIRST_VIOLATION,83.991519
INSPECTION_DEMERITS,11.442286
INSPECTION_GRADE,3.432436
RESTAURANT_LOCATION,0.586296
NUMBER_OF_VIOLATIONS,0.472934
ZIP,0.060621
INSPECTION_RESULT,0.011678
INSPECTION_TIME_HOUR,0.001387
CURRENT_DEMERITS,0.000471
PERMIT_STATUS,0.000247


**Key Takeaways from our analysis**



*   All though the City of Las Vegas has assumed huge amount of data would help in estimating the likelihhod of the Restaurant to get Critical Violation result it is actually dependent on few factors. The rest of the data would just add on to the noise in our modeling approach.
*   First Violation turns out to be the most important factor in determining Critical Violations for restaurants.
*  Inspection Demerits and Inspection Grade are also very important in determining Critical Violations
*   Lastly, Crtical Violation is also subject to the location in which the Restaurant belongs.



